## Optimizer: Finding best hyper-parameters

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install "numpy<1.24.0"

In [ ]:
pip install transformers datasets scikit-optimize

In [ ]:
pip install accelerate -U

In [ ]:
pip install transformers[torch]

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
import numpy as np
from sklearn.metrics import classification_report, accuracy_score
from transformers import TrainerCallback
import os
import shutil
import re
import time
from pathlib import Path

from skopt.space import Real, Categorical, Integer
from skopt.utils import use_named_args
import numpy as np
from skopt import gp_minimize

In [8]:
#Define the space of hyperparameters to search
search_space = [
  Real(1e-5, 5e-5, name='learning_rate'),
  Real(0.01, 0.2,  name='weight_decay'),
  Integer(3, 5, name='num_train_epochs'),
  Integer(8, 32, name='per_device_train_batch_size', prior='log-uniform'),
  Integer(8, 64, name='per_device_eval_batch_size', prior='log-uniform'),
]


@use_named_args(search_space)
def objective(learning_rate, num_train_epochs, per_device_train_batch_size, per_device_eval_batch_size, weight_decay):
    __file__ = "/content/drive/MyDrive/FinalProject/Models/ERNIE/ERNIE_optimizer.ipynb"
    file_name = "dataset_balanced_4000"
    ext ="xlsx"
    path_type = "Balanced"

    current_file_path = Path(__file__).parent
    path_to_project = current_file_path.parents[1]

    df = pd.read_excel(f"{path_to_project}/Data/Datasets/{path_type}/{file_name}.{ext}")

    results_dir = f"{path_to_project}/Models/ERNIE/Output/{path_type}/{file_name}"
    dump_dir = results_dir+"/Dump"

    if os.path.isdir(results_dir):
        shutil.rmtree(results_dir)

    os.mkdir(results_dir)
    os.mkdir(dump_dir)

    df = df[df['review'].notna() & (df['review'] != '')]
    # Select the text and label columns
    df['review'] = df['review'].str.replace('[^\x20-\x7E]', '', regex=True)
    X = df['review'].values
    y = df['label'].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


    # Encode the labels to a numeric format
    label_encoder = LabelEncoder()
    y_train_encoded = label_encoder.fit_transform(y_train)
    y_test_encoded = label_encoder.transform(y_test)

    # Initialize the tokenizer for RoBERTa
    tokenizer = AutoTokenizer.from_pretrained("nghuyong/ernie-2.0-base-en")
    # Tokenization function
    def tokenize_function(texts):
        return tokenizer(texts, padding="max_length", truncation=True, max_length=128)


    # Tokenize the data
    train_encodings = tokenize_function(X_train.tolist())
    val_encodings = tokenize_function(X_test.tolist())

    # Create dataset objects
    train_dataset = ReviewDataset(train_encodings, y_train_encoded)
    val_dataset = ReviewDataset(val_encodings, y_test_encoded)

    # Initialize the model for each fold
    model = AutoModelForSequenceClassification.from_pretrained("nghuyong/ernie-2.0-base-en", num_labels=len(label_encoder.classes_))


    # Define training arguments for each fold, adjust hyperparameters as needed
    training_args = TrainingArguments(
        output_dir=f"{dump_dir}/res",
        num_train_epochs= int(num_train_epochs),
        per_device_train_batch_size= int(per_device_train_batch_size),
        per_device_eval_batch_size= int(per_device_eval_batch_size),
        warmup_steps=500,
        weight_decay=weight_decay,
        logging_dir=f"{dump_dir}/logs",
        logging_strategy="epoch",
        evaluation_strategy="epoch",
        learning_rate=learning_rate,
        max_grad_norm=1.0,
        load_best_model_at_end=True,
        metric_for_best_model="accuracy",
        save_strategy="epoch",
        save_total_limit=2,
        lr_scheduler_type='linear'
    )

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=lambda p: {"accuracy": accuracy_score(p.predictions.argmax(-1), p.label_ids)}
        )

    # Train
    trainer.train()
    # Evaluate
    results = trainer.evaluate()
    neg_accuracy = -results['eval_accuracy']

    print(results['eval_accuracy'])
    print(neg_accuracy)

    shutil.rmtree(dump_dir)
    return neg_accuracy

# Custom dataset class
class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

result = gp_minimize(objective, search_space, n_calls=20, random_state=0)

print("Best parameters:")
print(result.x)




Some weights of ErnieForSequenceClassification were not initialized from the model checkpoint at nghuyong/ernie-2.0-base-en and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.123400,0.781327,0.705000
2,0.680300,0.693591,0.732500
3,0.534800,0.679082,0.741250
4,0.383500,0.790912,0.731250
5,0.200800,0.797438,0.750000


0.75
-0.75


Some weights of ErnieForSequenceClassification were not initialized from the model checkpoint at nghuyong/ernie-2.0-base-en and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.985600,0.737532,0.713750
2,0.642900,0.689280,0.742500
3,0.383700,0.738532,0.752500


...

Epoch,Training Loss,Validation Loss,Accuracy
1,1.010500,0.756128,0.701250
2,0.636600,0.655472,0.758750
3,0.473700,0.792423,0.721250
4,0.294400,0.936455,0.731250
5,0.122400,0.974524,0.751250


0.75875
-0.75875


Some weights of ErnieForSequenceClassification were not initialized from the model checkpoint at nghuyong/ernie-2.0-base-en and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.940700,0.789843,0.712500
2,0.653400,0.686732,0.758750
3,0.386000,0.867341,0.752500
4,0.214100,1.098068,0.758750
5,0.100900,1.190189,0.761250


0.76125
-0.76125
Best parameters:
[3.592687488204789e-05, 0.07996589256970411, 5, 10, 49]
